In [1]:
import numpy as np
import pandas as pd
import sklearn
import scipy

from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler

/opt/ml/.cache/pypoetry/virtualenvs/content-based_filtering-tF9KOkQL-py3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 원두정보메타데이터 재구성 - 전처리 및 Feature Engineering
- **상품명**을, 가공된 데이터인 merged_data 기준으로 변경
- Feature Engineering을 통해 각 아이템에 대한 **리뷰 수**, **전체 평점** 컬럼을 추가

### 원두정보메타데이터 vs. merged data

In [2]:
beans_data = pd.read_csv('../data/원두정보메타데이터.csv')
print(beans_data.shape)
beans_data.head(1)

(449, 12)


,Bean name,Cupping Note 향미,Cupping Note 산미,Cupping Note 단맛,Cupping Note 바디감,Cupping Note 밸런스,Rosting Point,origin,text,로스터리,validation,coupang link
0,베트남로부스타블루드래곤워시드G1,7.0,6,7,8,7.0,[시티/풀시티],베트남,로부스타 특유의 곡물의 구수한 향미와 은은한 단 맛이 느껴지는 커피입니다. 수세식(...,콩스콩스,O,https://www.coupang.com/vp/products/68982489?i...


In [3]:
beans_data[beans_data['로스터리'] == '콩스콩스'].shape

(42, 12)

In [4]:
merged_data = pd.read_csv('../data/merged_data.csv') # 콩스콩스만 리뷰가 있으므로 콩스콩스 원두 데이터만 포함
print(merged_data.shape)
merged_data.head(1)

(1280, 14)


,상품명,구매자 이름,구매자 평점,리뷰 제목,리뷰 내용,맛 만족도,Cupping Note 향미,Cupping Note 산미,Cupping Note 단맛,Cupping Note 바디감,Cupping Note 밸런스,Rosting Point,origin,text
0,아일리쉬향커피,이*혁,5,처음에 뭐지 싶었는데 먹다 보니 존맛이란 걸 알았다.,헤이즐넛 커피만 즐겨마시다 이건 또 무슨 맛일까 싶어서 사봤는데뭔가 향도 인공적인 ...,예상보다 맛있어요,8.0,6,6,7,6.0,[시티/풀시티],베트남,베트남 원두를 베이스로 아일리쉬 향을 가미하여 아일리쉬 특유의 위스키향과 크림 향미...


In [5]:
merged_data['상품명'].nunique()

32

merged data에는 리뷰가 없는 상품들 10개가 빠져 있음

### merged data 상품명 기준으로 상품명 변경

In [6]:
def change_bean_name(origin_bean_name):
    changed_bean_name = origin_bean_name.apply(lambda x: x.replace(" ", "")).apply(lambda x: x.lower())  # 띄어쓰기 없이 통일
    return changed_bean_name

In [7]:
def preprocess_beans_data(beans_data): 
    beans_data = beans_data.rename(columns={"Bean name": "상품명"})  # 정보 데이터의 컬럼명과 통일
    beans_data = beans_data.drop(columns=["validation", "coupang link"])  # 사용할 필요 없는 컬럼 제거

    # 콩스콩스 원두에 대해서만 상품명을 변경
    mask = beans_data["로스터리"] == "콩스콩스"
    beans_data.loc[mask, "상품명"] = change_bean_name(beans_data.loc[mask, "상품명"])
    
    return beans_data

In [8]:
beans_data = preprocess_beans_data(beans_data)
beans_data

,상품명,Cupping Note 향미,Cupping Note 산미,Cupping Note 단맛,Cupping Note 바디감,Cupping Note 밸런스,Rosting Point,origin,text,로스터리
0,베트남로부스타블루드래곤워시드g1,7.0,6,7,8,7.0,[시티/풀시티],베트남,로부스타 특유의 곡물의 구수한 향미와 은은한 단 맛이 느껴지는 커피입니다. 수세식(...,콩스콩스
1,마일드블렌드,7.0,7,7,7,7.0,[풀시티],"브라질, 베트남",묵직한 바디감과 부드러운 산미가 잘 조화되어 맛의 균형을 잘 살린 원두로써 가장 인...,콩스콩스
2,모카블렌드,6.0,6,6,7,6.0,[풀시티],"에티오피아, 브라질, 베트남",고소하면서도 부드러운 단 맛과 쌉싸름함이 어우러진 커피를 적절하게 블렌딩한 커피입니다.,콩스콩스
3,하우스블렌드,6.0,4,4,6,6.0,[풀시티],"브라질, 콜롬비아, 베트남",각 원두의 맛과 함미를 잘 배합하여 균형 잡힌 원두로 콩스콩스 블렌딩만의 노하우가 ...,콩스콩스
4,헤이즐넛향커피,8.0,6,6,7,6.0,[시티/풀시티],베트남,베트남 원두를 베이스로 헤이즐넛 향을 가미하여 헤이즐넛의 고소함과 풍부한 향이를 느...,콩스콩스
...,...,...,...,...,...,...,...,...,...,...
444,나비,NaN,8,10,8,NaN,미디엄 로스팅,"에티오피아, 콜롬비아","아카시아, 라일락 같은 꽃 향기가 아름다워요. 오렌지, 스피아민트 향미가 달고 맛있어요.",나무사이로
445,케냐 AA TOP 키안두 스페셜티 풀리워시드,NaN,8,8,6,NaN,미디엄 로스팅,케냐,"키안두 커피는 니에리 지역의 게냐산 해발 1,800m의 커피 재배에 최적의 조건을 ...",커스프 커피
446,다크우드,NaN,6,10,8,NaN,다크 로스팅,"과테말라, 에티오피아",다크우드 블렌드는 첼로 선율처럼 묵직하면서도 매끄럽고 오랫동안 입안을 맴도는 여운이...,나무사이로
447,윈터 블렌드,NaN,2,6,8,NaN,미디엄 다크 로스팅,"인도네시아, 브라질, 과테말라",이름에 걸맞게 추운 겨울 떠오르는 묵직한맛의 정석의 CUSP 블렌드 커피입니다. 와...,커스프 커피


In [9]:
mask = beans_data["로스터리"] == "콩스콩스"
beans_data.loc[mask, "상품명"]

0     베트남로부스타블루드래곤워시드g1
1                마일드블렌드
2                 모카블렌드
3                하우스블렌드
4               헤이즐넛향커피
5                 리치블렌드
6                아이스블렌드
7            에티오피아시다모g4
8           콜롬비아슈프리모후일라
9       브라질세하도ny-217-18
10             에스프레소블렌드
11               바닐라향커피
12              아일리쉬향커피
13                허니블렌드
14                스타블렌드
15               브라질블렌드
16             에티오피아블렌드
17              콜롬비아블렌드
18              시그니처블렌드
19                로얄블렌드
20            베트남아라비카g1
21          에티오피아예가체프g4
22             블루마운틴블렌드
23          에티오피아예가체프g2
24              파푸아뉴기니a
25        엘살바도르팬시shg워시드
26         멕시코알투라shg워시드
27              니카라과shg
28                모닝블렌드
29               스페셜블렌드
30              온두라스shg
31              과테말라shb
32              바리스타블렌드
33                 문블렌드
34             코스타리카shb
35          탄자니아킬리만자로aa
36           인도네시아만델링g1
37                 케냐aa
38             디카페인콜롬비아
39          디카페인에티오피아짐마
40             디카페인과테말라
41              

In [10]:
set(beans_data.loc[mask, "상품명"]) - set(merged_data['상품명'])

{'디카페인과테말라',
 '디카페인브라질',
 '디카페인에티오피아짐마',
 '로얄블렌드',
 '리치블렌드',
 '모닝블렌드',
 '시그니처블렌드',
 '에티오피아블렌드',
 '엘살바도르팬시shg워시드',
 '허니블렌드'}

리뷰가 없는 10개 원두만 나온 걸 확인함으로써, 원두정보메타데이터의 상품명을 merged_data 상품명 기준으로 올바르게 변경했음을 파악

### Feature Engineering
- 상품 전체 평점
- 상품 리뷰 수

In [11]:
merged_data_tmp = merged_data.groupby('상품명')['구매자 평점'].agg(['mean', 'count']).reset_index().rename(columns={'mean':'평점', 'count':'리뷰 수'})
merged_data_tmp 

,상품명,평점,리뷰 수
0,과테말라shb,4.666667,15
1,니카라과shg,3.000000,1
2,디카페인콜롬비아,3.000000,1
3,마일드블렌드,4.330383,339
4,멕시코알투라shg워시드,5.000000,2
5,모카블렌드,4.785714,14
6,문블렌드,4.727273,11
7,바닐라향커피,3.350000,20
8,바리스타블렌드,5.000000,1
9,베트남로부스타블루드래곤워시드g1,4.345098,255


In [12]:
beans_data_v2 = beans_data.merge(merged_data_tmp, how='left')
beans_data_v2

,상품명,Cupping Note 향미,Cupping Note 산미,Cupping Note 단맛,Cupping Note 바디감,Cupping Note 밸런스,Rosting Point,origin,text,로스터리,평점,리뷰 수
0,베트남로부스타블루드래곤워시드g1,7.0,6,7,8,7.0,[시티/풀시티],베트남,로부스타 특유의 곡물의 구수한 향미와 은은한 단 맛이 느껴지는 커피입니다. 수세식(...,콩스콩스,4.345098,255.0
1,마일드블렌드,7.0,7,7,7,7.0,[풀시티],"브라질, 베트남",묵직한 바디감과 부드러운 산미가 잘 조화되어 맛의 균형을 잘 살린 원두로써 가장 인...,콩스콩스,4.330383,339.0
2,모카블렌드,6.0,6,6,7,6.0,[풀시티],"에티오피아, 브라질, 베트남",고소하면서도 부드러운 단 맛과 쌉싸름함이 어우러진 커피를 적절하게 블렌딩한 커피입니다.,콩스콩스,4.785714,14.0
3,하우스블렌드,6.0,4,4,6,6.0,[풀시티],"브라질, 콜롬비아, 베트남",각 원두의 맛과 함미를 잘 배합하여 균형 잡힌 원두로 콩스콩스 블렌딩만의 노하우가 ...,콩스콩스,4.000000,6.0
4,헤이즐넛향커피,8.0,6,6,7,6.0,[시티/풀시티],베트남,베트남 원두를 베이스로 헤이즐넛 향을 가미하여 헤이즐넛의 고소함과 풍부한 향이를 느...,콩스콩스,4.250000,136.0
...,...,...,...,...,...,...,...,...,...,...,...,...
444,나비,NaN,8,10,8,NaN,미디엄 로스팅,"에티오피아, 콜롬비아","아카시아, 라일락 같은 꽃 향기가 아름다워요. 오렌지, 스피아민트 향미가 달고 맛있어요.",나무사이로,NaN,NaN
445,케냐 AA TOP 키안두 스페셜티 풀리워시드,NaN,8,8,6,NaN,미디엄 로스팅,케냐,"키안두 커피는 니에리 지역의 게냐산 해발 1,800m의 커피 재배에 최적의 조건을 ...",커스프 커피,NaN,NaN
446,다크우드,NaN,6,10,8,NaN,다크 로스팅,"과테말라, 에티오피아",다크우드 블렌드는 첼로 선율처럼 묵직하면서도 매끄럽고 오랫동안 입안을 맴도는 여운이...,나무사이로,NaN,NaN
447,윈터 블렌드,NaN,2,6,8,NaN,미디엄 다크 로스팅,"인도네시아, 브라질, 과테말라",이름에 걸맞게 추운 겨울 떠오르는 묵직한맛의 정석의 CUSP 블렌드 커피입니다. 와...,커스프 커피,NaN,NaN


### Roasting Point 단위 통일
- Roasting Point 표기가 한국식/일본식/SCAA 용어들이 모두 혼재되어 있음
- https://m.blog.naver.com/mingstar15/221318949761에 따라 단계별로 1~8 사이의 float으로 통일

In [13]:
beans_data_v2.rename(columns={'Rosting Point':'Roasting Point'}, inplace=True) # 스펠링에 오타가 있어 변경
beans_data_v2.head(1)

,상품명,Cupping Note 향미,Cupping Note 산미,Cupping Note 단맛,Cupping Note 바디감,Cupping Note 밸런스,Roasting Point,origin,text,로스터리,평점,리뷰 수
0,베트남로부스타블루드래곤워시드g1,7.0,6,7,8,7.0,[시티/풀시티],베트남,로부스타 특유의 곡물의 구수한 향미와 은은한 단 맛이 느껴지는 커피입니다. 수세식(...,콩스콩스,4.345098,255.0


In [14]:
beans_data_v2['Roasting Point'].value_counts()

Roasting Point
미디엄 다크 로스팅     110
미디엄 로스팅         90
[시티/풀시티]        85
라이트 미디엄 로스팅     52
다크 로스팅          43
라이트 로스팅         25
[풀시티]           16
[시티]             9
[중볶음]            5
[하이/시티]          4
[중강볶음]           4
[약볶음]            2
[중볶음/중강볶음]       1
[풀시티/프랜치]        1
[하이]             1
[플시티/프렌치]        1
Name: count, dtype: int64

In [15]:
transform_roasting_point = {
    '미디엄 다크 로스팅': 6,
    '미디엄 로스팅': 5,
    '[시티/풀시티]': 5.5,
    '라이트 미디엄 로스팅': 4,
    '다크 로스팅': 7,
    '라이트 로스팅': 2,
    '[풀시티]': 6,
    '[시티]': 5,
    '[중볶음]': 4,
    '[하이/시티]': 4.5,
    '[중강볶음]': 5,
    '[약볶음]': 2,
    '[중볶음/중강볶음]': 4.5,
    '[풀시티/프랜치]': 6.5,
    '[하이]': 4,
    '[플시티/프렌치]': 6.5
}

beans_data_v2['Roasting Point'] = beans_data_v2['Roasting Point'].map(transform_roasting_point)
beans_data_v2['Roasting Point'].value_counts()

Roasting Point
6.0    126
5.0    103
5.5     85
4.0     58
7.0     43
2.0     27
4.5      5
6.5      2
Name: count, dtype: int64

In [16]:
beans_data_v2.head()

,상품명,Cupping Note 향미,Cupping Note 산미,Cupping Note 단맛,Cupping Note 바디감,Cupping Note 밸런스,Roasting Point,origin,text,로스터리,평점,리뷰 수
0,베트남로부스타블루드래곤워시드g1,7.0,6,7,8,7.0,5.5,베트남,로부스타 특유의 곡물의 구수한 향미와 은은한 단 맛이 느껴지는 커피입니다. 수세식(...,콩스콩스,4.345098,255.0
1,마일드블렌드,7.0,7,7,7,7.0,6.0,"브라질, 베트남",묵직한 바디감과 부드러운 산미가 잘 조화되어 맛의 균형을 잘 살린 원두로써 가장 인...,콩스콩스,4.330383,339.0
2,모카블렌드,6.0,6,6,7,6.0,6.0,"에티오피아, 브라질, 베트남",고소하면서도 부드러운 단 맛과 쌉싸름함이 어우러진 커피를 적절하게 블렌딩한 커피입니다.,콩스콩스,4.785714,14.0
3,하우스블렌드,6.0,4,4,6,6.0,6.0,"브라질, 콜롬비아, 베트남",각 원두의 맛과 함미를 잘 배합하여 균형 잡힌 원두로 콩스콩스 블렌딩만의 노하우가 ...,콩스콩스,4.000000,6.0
4,헤이즐넛향커피,8.0,6,6,7,6.0,5.5,베트남,베트남 원두를 베이스로 헤이즐넛 향을 가미하여 헤이즐넛의 고소함과 풍부한 향이를 느...,콩스콩스,4.250000,136.0


### 업데이트 된 원두정보메타데이터 저장

In [17]:
beans_data_v2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449 entries, 0 to 448
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   상품명               449 non-null    object 
 1   Cupping Note 향미   129 non-null    float64
 2   Cupping Note 산미   449 non-null    int64  
 3   Cupping Note 단맛   449 non-null    int64  
 4   Cupping Note 바디감  449 non-null    int64  
 5   Cupping Note 밸런스  95 non-null     float64
 6   Roasting Point    449 non-null    float64
 7   origin            428 non-null    object 
 8   text              449 non-null    object 
 9   로스터리              449 non-null    object 
 10  평점                33 non-null     float64
 11  리뷰 수              33 non-null     float64
dtypes: float64(5), int64(3), object(4)
memory usage: 42.2+ KB


In [18]:
beans_data_v2.to_csv('../data/원두정보메타데이터_v2.csv', index=False)

## 상품 설명 텍스트에서 키워드 추출

In [19]:
data = pd.read_csv('../data/원두정보메타데이터_v2.csv')

### 키워드를 단어 단위로 추출

In [20]:
kiwi = Kiwi()

# 명사 추출 함수
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N') or pos.startswith('SL'):
            results.append(token)
    return results

In [21]:
nouns_list = []
for text in data['text']:
    nouns = noun_extractor(text)
    nouns_list.append(nouns)

print(nouns_list[0])

['스타', '특유', '곡물', '향미', '커피', '수세식', '워시드', '가공', '방식', '가공', '균일', '에프터', '테이스트', '선사']


In [22]:
text_list = []
for nouns in nouns_list:
    text = ' '.join(nouns) # 리스트에 담긴 각 단어들을 이어 문자열 형태로 바꿈
    text_list.append(text)
    
print(text_list[0])

스타 특유 곡물 향미 커피 수세식 워시드 가공 방식 가공 균일 에프터 테이스트 선사


In [23]:
# 키워드 추출
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)

keywords_list = []
for text in text_list:
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
    keywords_list.append(keywords)

print(len(keywords_list))

449


### 필요 없는 키워드 제외

In [24]:
keywords_list

[[('균일', 0.6621),
  ('향미', 0.6298),
  ('가공', 0.6253),
  ('테이스트', 0.5877),
  ('워시드', 0.5791),
  ('에프터', 0.5457),
  ('스타', 0.5369),
  ('수세식', 0.5278),
  ('선사', 0.5034),
  ('방식', 0.4928),
  ('커피', 0.4841),
  ('곡물', 0.4715),
  ('특유', 0.4141)],
 [('상품', 0.6803),
  ('산미', 0.5182),
  ('원두', 0.503),
  ('조화', 0.5002),
  ('바디감', 0.4978),
  ('균형', 0.4867),
  ('인기', 0.4542)],
 [('커피', 0.845), ('쌉싸름', 0.6844), ('블렌딩', 0.566)],
 [('콩스콩스', 0.7686),
  ('원두', 0.7096),
  ('블렌딩', 0.6685),
  ('배합', 0.5896),
  ('함미', 0.563),
  ('노하우', 0.5409),
  ('커피', 0.4351),
  ('균형', 0.3243)],
 [('헤이즐넛', 0.7901),
  ('원두', 0.6755),
  ('베이스', 0.6492),
  ('가미', 0.6332),
  ('커피', 0.5364),
  ('베트남', 0.4089)],
 [('쌀싸름이', 0.7458),
  ('커피', 0.6805),
  ('조화', 0.4851),
  ('블렌딩', 0.4514),
  ('바디', 0.4427),
  ('균형', 0.3175)],
 [('블렌딩', 0.7253),
  ('아이스', 0.673),
  ('향미', 0.6591),
  ('커피', 0.526),
  ('여름', 0.3845)],
 [('커피', 0.7708),
  ('향미', 0.62),
  ('와인', 0.5939),
  ('산미', 0.4517),
  ('베리류', 0.4458),
  ('바디감', 0.4352),
  ('약간', 0

In [25]:
all_keywords_list = []
for keywords in keywords_list:
    res = [keyword for keyword, _ in keywords]
    all_keywords_list.extend(res)
    
print(len(all_keywords_list)) # 모든 키워드들의 등장 횟수 (중복 O)
print(all_keywords_list)

4048
['균일', '향미', '가공', '테이스트', '워시드', '에프터', '스타', '수세식', '선사', '방식', '커피', '곡물', '특유', '상품', '산미', '원두', '조화', '바디감', '균형', '인기', '커피', '쌉싸름', '블렌딩', '콩스콩스', '원두', '블렌딩', '배합', '함미', '노하우', '커피', '균형', '헤이즐넛', '원두', '베이스', '가미', '커피', '베트남', '쌀싸름이', '커피', '조화', '블렌딩', '바디', '균형', '블렌딩', '아이스', '향미', '커피', '여름', '커피', '향미', '와인', '산미', '베리류', '바디감', '약간', '견과', '크림', '커피', '부담', '누구', '커피', '향미', '브라질', '처음', '아몬드', '특징', '표현', '커피', '에스프레소', '조화', '블렌딩', '아로아향', '한미', '커피', '기분', '느낌', '아일리쉬', '원두', '향미', '크림', '가미', '베이스', '커피', '위스키', '베트남', '특유', '쌉싸름', '향미', '커피', '블렌딩', '바디감', '커피', '산미', '인기', '대중', '커피', '단맛', '아몬드', '커피', '쌉싸름', '산미', '향미', '초콜릿', '커피', '아몬드', '산미', '약간', '다크', '부담', '쌉싸름', '향미', '커피', '블렌딩', '누구', '커피', '쌉싸름', '누구', '향미', '커피', '각종', '깔끔', '베리', '후미', '바디', '감칠맛', '커피', '단맛', '조화', '블렌딩', '산미', '완성도', '초콜릿', '레몬', '향미', '커피', '그라스', '자두', '인상', '시트러스', '후미', '밀크', '감칠맛', '향미', '고급', '커피', '풍미', '커피', '마일드', '기분', '초콜릿', '커피', '산미', '바디', '밀크', '향미', '테이스트', 

In [26]:
all_keywords_cnt = pd.Series(all_keywords_list).value_counts()
print(all_keywords_cnt)

커피     267
단맛     195
산미     124
초콜릿     92
향미      80
      ... 
역할       1
화성       1
수원       1
스쿨       1
핸드       1
Name: count, Length: 941, dtype: int64


전체 고유 키워드 개수는 941개

In [27]:
# 10번 이하 등장하는 (자주 등장하지 않는) 키워드들
for i in range(10): # 1번 ~ 10번 등장하는 키워드들을 차례대로 확인
    i_keyword_list = all_keywords_cnt[all_keywords_cnt==i+1].index.tolist() # i번 등장하는 키워드 리스트
    print(f'-----------{i+1}-----------')
    print(len(i_keyword_list))
    print(i_keyword_list)

-----------1-----------
531
['사장', '더블', '원종', '다재다능', '수분', '해발', '콤롬비아', '소화', '쿠아', '기간', '선명', '상권', '키안', '마이크로랏', '후일라', '비센테', '플라타', '전형', '제너럴', '플루소르', '보사노바', '블렌드커피로', '존재감', '완성', '형태', '주변', '레지오날', '그럼블', '간직', '한쪽', '포용력', '마을', '상황', '스피아민트', '아카시아', '스파클링', '국화꽃', '라일락', '허브', '슈가케', '그레이드', '베르', '니에리', '인간', 'coffee', '키비리기', '재배고도', '성향', 'kibirigwi', '물음', '발전', '출발', '길링바사', 'g1', '이성', '감성', '트리플', '여정', '키린야가', 'bright', '매칭', '컨셉', 'sweet', '무엇', 'grader', '초콜레', '누텔라', '프로파일', '무화과', '안녕', '탄소', '활용', 'cooperative', 'ruiru', '관리', '아메이카노', '조건', '현대', '첼로', 'forest', '선율', '사치', '집중', '새소리', '마일', '송버드', '한라봉', '푸루티', '다크우드', '인과', '생산자', '로즈마리', '극대', '캔버스', '마우', '로아', '플로럴', '라떼와', 'kirinyaga', 'sl', '라이', '협동조합', 'farmers', '벽난로', 'day', '솔티드', '근로자', '감촉', 'henrique', '경력', '과즙', '여름날', '계절', '실크', '커피로스터스의', '취향', '실키', '스탠딩', '공기', '웨하스', '기반', '과자', '알레그레', '소코로', '다년간', '소유', '엿기름', '로렌', '수상', '기술', '농업', '케인슈가', '로즈워터', '크래커', '베리잼', '식감', '치즈', '향

- 한 번만 등장하는 키워드만 무려 531개
- 두 번 등장하는 키워드까지는 필요 없어 보이는 게 많아, 2번 이하 등장하는 키워드들은 모두 제외해도 될 것 같음

In [28]:
# 10번 이상 등장하는 (자주 등장하는) 키워드들
print(all_keywords_cnt[all_keywords_cnt>10].index.tolist()) 

['커피', '단맛', '산미', '초콜릿', '향미', '바디', '견과', '블렌드', '다크', '특징', '여운', '풍미', '과일', '원두', '밸런스', '블렌딩', '조화', '쌉싸름', '매력', '오렌지', '카라멜', '기분', '바디감', '카페인', '느낌', '추천', '로스팅', '아몬드', '설탕', '밀크', '특유', '누구', '균형', '균형감', '에스프레소', '질감', '쓴맛', '신맛', '생두', '후미', '편안', '레몬', '베이스', '에티오피아', '감칠맛', '초콜렛', '베리', '깔끔', '카카오', '향기', '스페셜', '복합', '부담', '감귤', '브라질', '표현', '대중', '시작', '라떼']


위 단어들을 살펴봤을 때,
'커피', '특징', '원두', '기분', '느낌', '추천', '누구', '표현', '약간', '이름', '사용', '제거', '배전', '적합', '지역', '유지', '사이', '초점', '가지', '상품', '각종', '추출', '모금', '우리', '정도', '세팅', '최대한'
=> 커피 특징을 나타내는 데 도움이 되지 않는 단어들이므로 해당 단어들은 제외

In [29]:
# 아이템 특징과 관련 없어 보이는 키워드를 직접 선정
no_need_keywords = ['커피', '특징', '원두', '기분', '느낌', '추천', '누구', '표현', '약간', 
                    '이름', '사용', '제거', '배전', '적합', '지역', '유지', '사이', '초점', 
                    '가지', '상품', '각종', '추출', '모금', '우리', '정도', '세팅', '최대한']

In [30]:
# 영어로 이루어진 키워드 & 등장횟수 확인
print([(keyword, all_keywords_cnt[keyword]) for keyword in all_keywords_cnt.index if all(ord(char) < 128 for char in keyword)])

# 숫자를 포함하는 키워드 & 등장횟수 확인
print([(keyword, all_keywords_cnt[keyword]) for keyword in all_keywords_cnt.index if any(char.isdigit() for char in keyword)])

[('felt', 4), ('cusp', 4), ('csp', 2), ('tea', 2), ('diploma', 2), ('sca', 2), ('cqi', 2), ('aa', 2), ('coffee', 1), ('kibirigwi', 1), ('g1', 1), ('bright', 1), ('sweet', 1), ('grader', 1), ('cooperative', 1), ('ruiru', 1), ('forest', 1), ('kirinyaga', 1), ('sl', 1), ('farmers', 1), ('day', 1), ('henrique', 1), ('espresso', 1), ('wakan', 1), ('we', 1), ('ethiopia', 1), ('may', 1), ('moonlight', 1), ('life', 1), ('good', 1), ('so', 1), ('variation', 1), ('aftertaste', 1), ('bittersweet', 1), ('nightfall', 1), ('roasting', 1), ('ice', 1), ('midium', 1), ('oz', 1), ('or', 1), ('peaberry', 1), ('nswf', 1), ('snug', 1), ('house', 1), ('bacs', 1), ('goodies', 1), ('oldies', 1), ('but', 1)]
[('g1', 1), ('2인자', 1)]


In [31]:
# 영어 혹은 숫자를 포함하는 키워드들 저장
eng_digit_keywords_list = [keyword for keyword in all_keywords_cnt.index if any(ord(char) < 128 or char.isdigit() for char in keyword)]
print(len(eng_digit_keywords_list))
print(eng_digit_keywords_list)

49
['felt', 'cusp', 'csp', 'tea', 'diploma', 'sca', 'cqi', 'aa', 'coffee', 'kibirigwi', 'g1', 'bright', 'sweet', 'grader', 'cooperative', 'ruiru', 'forest', 'kirinyaga', 'sl', 'farmers', 'day', 'henrique', 'espresso', 'wakan', 'we', 'ethiopia', 'may', 'moonlight', 'life', 'good', 'so', 'variation', 'aftertaste', 'bittersweet', 'nightfall', 'roasting', 'ice', 'midium', 'oz', '2인자', 'or', 'peaberry', 'nswf', 'snug', 'house', 'bacs', 'goodies', 'oldies', 'but']


- 영어 또는 숫자를 포함하는 키워드 대부분 등장 횟수도 적고 필요 없는 키워드가 많아서 모두 제외시켜도 될 것 같음

In [32]:
# 제외할 키워드 리스트 - (1) 2번 이하 등장하거나, (2) 커피 특징과 관련 없는 키워드거나, (3) 영어나 숫자를  포함하는 키워드들 
remove_keywords_list = list(set(all_keywords_cnt[all_keywords_cnt<=2].index.tolist() + no_need_keywords + eng_digit_keywords_list))
print(len(remove_keywords_list))
print(remove_keywords_list)

713
['슈가리', '토마티요', '전문', '달보드레', '공기', '행복감', 'kirinyaga', '추천', '에이드', '스탠딩', '시대', '흑맥주', 'moonlight', '제공', '올드', '귀부인', 'cusp', '우리말', '구름', '치즈', '전문가', '공작', '적합', '습식', '단조', '베러', '일품', '카우카', '연출', '씨리얼', '단골', '라일락', '제너럴', '껍질', '쿠아', '송버드', '하루스', '구매', '사치', '국내', '발생', '적정', '동일', '실크', '사이즈', '쌉살', '실키', 'kibirigwi', '월넛', '벚꽃', 'but', '이상', '달콤쌉싸름함이', '스푼', 'coffee', '이사벨', '마시멜로', '패키징', '그레이', '핀카', '협동조합', '토피넛', '시장', '마법', '주원료', '상급', '라임', '프로파일', '소유', '주도', '머신', '우리', '균일', '존재감', 'tea', '임산부', '군밤', '석류', '무엇', '무게', '모닝', '제주', '표현', '라이', '쥬스', '로즈마리', '블렌드예요', '안정', '버터리', '산업', '노란색', '수마트라', '머스캣', '노고', '리버벨', '피니쉬', '시간', '수분', '비센테', '특징', '이름', '오일', '웨이브온', '스카치', '기법', '중강', '파노라마', '프레스', '초밥', '약간', '오크', '셀러', '감성', '웨하스', '여정', '산타', '여기', '발전', '베이컨', '만다린', '가지', '망고스틴', '라이트', '마이소르', '박스프레소', '누구', '부스', '납품', 'aa', '마이크로', '독보', '모티브', '특성', '중년', '달고나', '이국', '엑셀', '누룽지', '이태리', '풍성', '메이', '만능', '브니엘', '뀬형', 'bacs', '너머', '와이칸과', '대비', 

In [33]:
len([x for x in all_keywords_cnt.index.tolist() if x not in remove_keywords_list]) # 941 - 713 = 228

228

In [34]:
description_keywords_list = []
description_keywords_sentence = []

for keywords in keywords_list:
    # 키워드를 집합으로 저장
    keywords_set = {keyword for keyword, _ in keywords if keyword not in remove_keywords_list}
    description_keywords_list.append(keywords_set)

    # 키워드만 뽑아 공백으로 구분하여 하나의 문자열로 합치기
    keywords_sentence = ' '.join([keyword for keyword, _ in keywords if keyword not in remove_keywords_list])
    description_keywords_sentence.append(keywords_sentence)

    # print(keywords_set, keywords_sentence)

In [35]:
data['keywords_set'] = description_keywords_list
data['keywords_sentence'] = description_keywords_sentence
data

,상품명,Cupping Note 향미,Cupping Note 산미,Cupping Note 단맛,Cupping Note 바디감,Cupping Note 밸런스,Roasting Point,origin,text,로스터리,평점,리뷰 수,keywords_set,keywords_sentence
0,베트남로부스타블루드래곤워시드g1,7.0,6,7,8,7.0,5.5,베트남,로부스타 특유의 곡물의 구수한 향미와 은은한 단 맛이 느껴지는 커피입니다. 수세식(...,콩스콩스,4.345098,255.0,"{워시드, 방식, 가공, 향미, 곡물, 특유, 선사}",향미 가공 워시드 선사 방식 곡물 특유
1,마일드블렌드,7.0,7,7,7,7.0,6.0,"브라질, 베트남",묵직한 바디감과 부드러운 산미가 잘 조화되어 맛의 균형을 잘 살린 원두로써 가장 인...,콩스콩스,4.330383,339.0,"{산미, 인기, 균형, 바디감, 조화}",산미 조화 바디감 균형 인기
2,모카블렌드,6.0,6,6,7,6.0,6.0,"에티오피아, 브라질, 베트남",고소하면서도 부드러운 단 맛과 쌉싸름함이 어우러진 커피를 적절하게 블렌딩한 커피입니다.,콩스콩스,4.785714,14.0,"{블렌딩, 쌉싸름}",쌉싸름 블렌딩
3,하우스블렌드,6.0,4,4,6,6.0,6.0,"브라질, 콜롬비아, 베트남",각 원두의 맛과 함미를 잘 배합하여 균형 잡힌 원두로 콩스콩스 블렌딩만의 노하우가 ...,콩스콩스,4.000000,6.0,"{배합, 블렌딩, 균형, 노하우}",블렌딩 배합 노하우 균형
4,헤이즐넛향커피,8.0,6,6,7,6.0,5.5,베트남,베트남 원두를 베이스로 헤이즐넛 향을 가미하여 헤이즐넛의 고소함과 풍부한 향이를 느...,콩스콩스,4.250000,136.0,"{베트남, 가미, 헤이즐넛, 베이스}",헤이즐넛 베이스 가미 베트남
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,나비,NaN,8,10,8,NaN,5.0,"에티오피아, 콜롬비아","아카시아, 라일락 같은 꽃 향기가 아름다워요. 오렌지, 스피아민트 향미가 달고 맛있어요.",나무사이로,NaN,NaN,"{오렌지, 향기, 향미}",향기 향미 오렌지
445,케냐 AA TOP 키안두 스페셜티 풀리워시드,NaN,8,8,6,NaN,5.0,케냐,"키안두 커피는 니에리 지역의 게냐산 해발 1,800m의 커피 재배에 최적의 조건을 ...",커스프 커피,NaN,NaN,"{살구, 오렌지, 노트, 스페셜, 케냐, 지속, 복숭아, 자몽, 여운, 최적}",복숭아 여운 살구 오렌지 스페셜 케냐 최적 노트 지속 자몽
446,다크우드,NaN,6,10,8,NaN,7.0,"과테말라, 에티오피아",다크우드 블렌드는 첼로 선율처럼 묵직하면서도 매끄럽고 오랫동안 입안을 맴도는 여운이...,나무사이로,NaN,NaN,"{여운, 단맛, 블렌드}",여운 블렌드 단맛
447,윈터 블렌드,NaN,2,6,8,NaN,6.0,"인도네시아, 브라질, 과테말라",이름에 걸맞게 추운 겨울 떠오르는 묵직한맛의 정석의 CUSP 블렌드 커피입니다. 와...,커스프 커피,NaN,NaN,"{메인, 추구, 카페, 다크, 블렌드로, 블렌드, 와인}",카페 블렌드로 와인 블렌드 메인 다크 추구


## 결측치 처리
이후 유사도를 계산하기 위해서는 결측치를 처리해줘야 함

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449 entries, 0 to 448
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   상품명                449 non-null    object 
 1   Cupping Note 향미    129 non-null    float64
 2   Cupping Note 산미    449 non-null    int64  
 3   Cupping Note 단맛    449 non-null    int64  
 4   Cupping Note 바디감   449 non-null    int64  
 5   Cupping Note 밸런스   95 non-null     float64
 6   Roasting Point     449 non-null    float64
 7   origin             428 non-null    object 
 8   text               449 non-null    object 
 9   로스터리               449 non-null    object 
 10  평점                 33 non-null     float64
 11  리뷰 수               33 non-null     float64
 12  keywords_set       449 non-null    object 
 13  keywords_sentence  449 non-null    object 
dtypes: float64(5), int64(3), object(6)
memory usage: 49.2+ KB


결측치 처리가 필요한 컬럼 => Cupping Note 향미, Cupping Note 밸런스, 평점, 리뷰 수 => 근데 다 결측치가 반 이상으로 너무 많음.. 쓸 수 있을까? 

### Cupping Note 향미

In [37]:
data['Cupping Note 향미'].median()

7.0

In [38]:
data[data['keywords_set'].apply(lambda x: '향미' in x)]['Cupping Note 향미'].median() # 키워드에 '향미'를 포함하는 경우 향미 수치의 평균

7.0

향미가 키워드로 언급된다고 해서 향미 값이 더 높지는 않음. 즉 키워드 정보로 유추할 수 없으므로 향미 피쳐는 따로 사용하는 것이 좋아 보임

In [39]:
# 중앙값으로 대체
data.loc[data['Cupping Note 향미'].isna(), 'Cupping Note 향미'] = data['Cupping Note 향미'].median()

### Cupping Note 밸런스

In [40]:
data['Cupping Note 밸런스'].mean()

6.673684210526316

In [41]:
data[data['keywords_set'].apply(lambda x: '밸런스' in x)]['Cupping Note 밸런스'].mean() # 키워드에 '밸런스'를 포함하는 경우 밸런스 수치의 평균

7.75

밸런스가 키워드에 언급되는 경우, 실제 밸런스 값도 더 높음. 키워드 정보와 어느정도 관련이 있는 것으로 판단되며 결측치가 향미 컬럼보다도 많으므로 이 밸런스 컬럼은 제외해도 될 것 같음

### 평점 & 리뷰 수

In [42]:
data[(data['로스터리'] == '콩스콩스') & (data['리뷰 수'].isna())].shape

(10, 14)

In [43]:
# 콩스콩스 로스터리에서 리뷰 수가 결측인 경우는 리뷰가 없었기 때문이므로, 해당 경우는 평점과 리뷰 수를 모두 0으로 대체

# 리뷰가 없는 콩스콩스 상품들의 인덱스
filter_idx = data[(data['로스터리'] == '콩스콩스') & (data['리뷰 수'].isna())].index
data.loc[filter_idx, '리뷰 수'] = 0
data.loc[filter_idx, '평점'] = 0

In [44]:
# 나머지 리뷰 수 및 평점 결측치들은 모두 중앙값으로 대체
print(data['리뷰 수'].median())
print(data['평점'].median())

data.loc[data['리뷰 수'].isna(), '리뷰 수'] = data['리뷰 수'].median()
data.loc[data['평점'].isna(), '평점'] = data['평점'].median()

6.0
4.330383480825959


In [45]:
# 평점을 소수 둘째 자리까지만 표시
data['평점'] = data['평점'].round(2)

# 리뷰 수는 정수형으로 변환
data['리뷰 수'] = data['리뷰 수'].map(lambda x: int(x))

data.head()

,상품명,Cupping Note 향미,Cupping Note 산미,Cupping Note 단맛,Cupping Note 바디감,Cupping Note 밸런스,Roasting Point,origin,text,로스터리,평점,리뷰 수,keywords_set,keywords_sentence
0,베트남로부스타블루드래곤워시드g1,7.0,6,7,8,7.0,5.5,베트남,로부스타 특유의 곡물의 구수한 향미와 은은한 단 맛이 느껴지는 커피입니다. 수세식(...,콩스콩스,4.35,255,"{워시드, 방식, 가공, 향미, 곡물, 특유, 선사}",향미 가공 워시드 선사 방식 곡물 특유
1,마일드블렌드,7.0,7,7,7,7.0,6.0,"브라질, 베트남",묵직한 바디감과 부드러운 산미가 잘 조화되어 맛의 균형을 잘 살린 원두로써 가장 인...,콩스콩스,4.33,339,"{산미, 인기, 균형, 바디감, 조화}",산미 조화 바디감 균형 인기
2,모카블렌드,6.0,6,6,7,6.0,6.0,"에티오피아, 브라질, 베트남",고소하면서도 부드러운 단 맛과 쌉싸름함이 어우러진 커피를 적절하게 블렌딩한 커피입니다.,콩스콩스,4.79,14,"{블렌딩, 쌉싸름}",쌉싸름 블렌딩
3,하우스블렌드,6.0,4,4,6,6.0,6.0,"브라질, 콜롬비아, 베트남",각 원두의 맛과 함미를 잘 배합하여 균형 잡힌 원두로 콩스콩스 블렌딩만의 노하우가 ...,콩스콩스,4.00,6,"{배합, 블렌딩, 균형, 노하우}",블렌딩 배합 노하우 균형
4,헤이즐넛향커피,8.0,6,6,7,6.0,5.5,베트남,베트남 원두를 베이스로 헤이즐넛 향을 가미하여 헤이즐넛의 고소함과 풍부한 향이를 느...,콩스콩스,4.25,136,"{베트남, 가미, 헤이즐넛, 베이스}",헤이즐넛 베이스 가미 베트남


### 모델 인풋에서 제외할 피쳐들

- 위처럼 결측치를 채우고 평점과 리뷰 수를 포함해 돌려봤을 때, 추천 결과가 좀 더 나빠졌음
- 현재 데이터에 콩스콩스만 해당 피쳐의 값을 가지고 있어 449개 중 33개를 제외하고는 결측치였기 때문에, 두 컬럼은 아예 제외시키는 게 좋아보임

In [46]:
drop_features = [
    '상품명',
    # 'Cupping Note 향미', 
    # 'Cupping Note 산미', 
    # 'Cupping Note 단맛', 
    # 'Cupping Note 바디감', 
    'Cupping Note 밸런스', 
    # 'Roasting Point', 
    'origin',
    'text',
    '로스터리',
    '평점', 
    '리뷰 수',
    'keywords_set',
    'keywords_sentence'
    ]

In [47]:
# data['origin'].fillna("", inplace=True) # 원산지(origin) 컬럼에 NaN 값이 있으면 연산 불가

## 텍스트 데이터 처리 방법 A) TF-IDF 벡터화

In [48]:
vectorizer = TfidfVectorizer()

tfidf_matrix = vectorizer.fit_transform(data['keywords_sentence']).toarray() # (data['keywords_sentence'] + " " + data['origin']).toarray()
tfidf_feature_names = vectorizer.get_feature_names_out()

원산지 컬럼은 우선 사용 x

In [49]:
tfidf_matrix = pd.DataFrame(tfidf_matrix, columns=tfidf_feature_names, index = data['상품명'])
print(tfidf_matrix.shape)
tfidf_matrix.head()

(449, 228)


,가공,가미,갈색,감귤,감칠맛,강조,개성,건자두,건포도,견과,...,허니,헤이즐넛,호두,호불호,홍차,화산,화이트,황설탕,후미,휴식
상품명,,,,,,,,,,,,,,,,,,,,,
베트남로부스타블루드래곤워시드g1,0.394962,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
마일드블렌드,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
모카블렌드,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
하우스블렌드,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
헤이즐넛향커피,0.000000,0.555504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.485493,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 나머지 아이템 피쳐 추가
위에 있는 건 텍스트 데이터로부터만 나온 피쳐들

In [50]:
item_features = [f for f in data.columns if f not in drop_features]

item_numeric_features_df = data[item_features].set_index(data['상품명'])
item_numeric_features_df

,Cupping Note 향미,Cupping Note 산미,Cupping Note 단맛,Cupping Note 바디감,Roasting Point
상품명,,,,,
베트남로부스타블루드래곤워시드g1,7.0,6,7,8,5.5
마일드블렌드,7.0,7,7,7,6.0
모카블렌드,6.0,6,6,7,6.0
하우스블렌드,6.0,4,4,6,6.0
헤이즐넛향커피,8.0,6,6,7,5.5
...,...,...,...,...,...
나비,7.0,8,10,8,5.0
케냐 AA TOP 키안두 스페셜티 풀리워시드,7.0,8,8,6,5.0
다크우드,7.0,6,10,8,7.0


In [51]:
item_matrix_A = pd.concat([tfidf_matrix, item_numeric_features_df], axis=1)
item_matrix_A

,가공,가미,갈색,감귤,감칠맛,강조,개성,건자두,건포도,견과,...,화산,화이트,황설탕,후미,휴식,Cupping Note 향미,Cupping Note 산미,Cupping Note 단맛,Cupping Note 바디감,Roasting Point
상품명,,,,,,,,,,,,,,,,,,,,,
베트남로부스타블루드래곤워시드g1,0.394962,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,6,7,8,5.5
마일드블렌드,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,7,7,7,6.0
모카블렌드,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,6,6,7,6.0
하우스블렌드,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,6.0,4,4,6,6.0
헤이즐넛향커피,0.000000,0.555504,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,8.0,6,6,7,5.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
나비,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,8,10,8,5.0
케냐 AA TOP 키안두 스페셜티 풀리워시드,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,8,8,6,5.0
다크우드,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,7.0,6,10,8,7.0


In [52]:
item_matrix_A.isna().sum().sum()

0

## 텍스트 데이터 처리 방법 B) 자카드 유사도 구하기
tfidf matrix 차원이 너무 크므로, 타깃 상품의 키워드와 다른 상품의 키워드들에 대한 자카드 유사도를 계산해 이를 한 컬럼으로 사용해보자는 아이디어

In [53]:
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    
    if union == 0:
        return 0  # 분모가 0인 경우 자카드 유사도를 0으로 반환
    
    similarity = intersection / union
    return similarity

In [54]:
target = data[data['상품명'] == '베트남로부스타블루드래곤워시드g1'] # target item 지정 필요
target['keywords_set'][0] # test

{'가공', '곡물', '방식', '선사', '워시드', '특유', '향미'}

In [55]:
# 자카드 유사도 계산
data['Keywords Jaccard Similarity'] = data.apply(lambda row: jaccard_similarity(set(target['keywords_set'][0]), set(row['keywords_set'])), axis=1)
data.head()

,상품명,Cupping Note 향미,Cupping Note 산미,Cupping Note 단맛,Cupping Note 바디감,Cupping Note 밸런스,Roasting Point,origin,text,로스터리,평점,리뷰 수,keywords_set,keywords_sentence,Keywords Jaccard Similarity
0,베트남로부스타블루드래곤워시드g1,7.0,6,7,8,7.0,5.5,베트남,로부스타 특유의 곡물의 구수한 향미와 은은한 단 맛이 느껴지는 커피입니다. 수세식(...,콩스콩스,4.35,255,"{워시드, 방식, 가공, 향미, 곡물, 특유, 선사}",향미 가공 워시드 선사 방식 곡물 특유,1.0
1,마일드블렌드,7.0,7,7,7,7.0,6.0,"브라질, 베트남",묵직한 바디감과 부드러운 산미가 잘 조화되어 맛의 균형을 잘 살린 원두로써 가장 인...,콩스콩스,4.33,339,"{산미, 인기, 균형, 바디감, 조화}",산미 조화 바디감 균형 인기,0.0
2,모카블렌드,6.0,6,6,7,6.0,6.0,"에티오피아, 브라질, 베트남",고소하면서도 부드러운 단 맛과 쌉싸름함이 어우러진 커피를 적절하게 블렌딩한 커피입니다.,콩스콩스,4.79,14,"{블렌딩, 쌉싸름}",쌉싸름 블렌딩,0.0
3,하우스블렌드,6.0,4,4,6,6.0,6.0,"브라질, 콜롬비아, 베트남",각 원두의 맛과 함미를 잘 배합하여 균형 잡힌 원두로 콩스콩스 블렌딩만의 노하우가 ...,콩스콩스,4.00,6,"{배합, 블렌딩, 균형, 노하우}",블렌딩 배합 노하우 균형,0.0
4,헤이즐넛향커피,8.0,6,6,7,6.0,5.5,베트남,베트남 원두를 베이스로 헤이즐넛 향을 가미하여 헤이즐넛의 고소함과 풍부한 향이를 느...,콩스콩스,4.25,136,"{베트남, 가미, 헤이즐넛, 베이스}",헤이즐넛 베이스 가미 베트남,0.0


### 나머지 아이템 피쳐들과 결합

In [56]:
data['Keywords Jaccard Similarity']

0      1.000000
1      0.000000
2      0.000000
3      0.000000
4      0.000000
         ...   
444    0.111111
445    0.000000
446    0.000000
447    0.000000
448    0.055556
Name: Keywords Jaccard Similarity, Length: 449, dtype: float64

In [57]:
# 위에서 계산한 자카드 유사도 컬럼과, 나머지 필요한 아이템 피쳐들을 결합
item_matrix_B = item_numeric_features_df.copy()
item_matrix_B['Keywords Jaccard Similarity'] = data['Keywords Jaccard Similarity'].values
item_matrix_B

,Cupping Note 향미,Cupping Note 산미,Cupping Note 단맛,Cupping Note 바디감,Roasting Point,Keywords Jaccard Similarity
상품명,,,,,,
베트남로부스타블루드래곤워시드g1,7.0,6,7,8,5.5,1.000000
마일드블렌드,7.0,7,7,7,6.0,0.000000
모카블렌드,6.0,6,6,7,6.0,0.000000
하우스블렌드,6.0,4,4,6,6.0,0.000000
헤이즐넛향커피,8.0,6,6,7,5.5,0.000000
...,...,...,...,...,...,...
나비,7.0,8,10,8,5.0,0.111111
케냐 AA TOP 키안두 스페셜티 풀리워시드,7.0,8,8,6,5.0,0.000000
다크우드,7.0,6,10,8,7.0,0.000000


## 정규화
피쳐별로 단위가 다르므로 min-max 정규화 진행

In [58]:
def scaling(item_matrix):
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(item_matrix)
    item_matrix = pd.DataFrame(scaled, columns=item_matrix.columns)
    return item_matrix

In [59]:
item_matrix_A = scaling(item_matrix_A) # 방법 A
item_matrix_B = scaling(item_matrix_B) # 방법 B

In [60]:
# item_matrix.describe()

## 코사인 유사도 구하기

In [61]:
# 방법 A
cosine_sim_A = cosine_similarity(item_matrix_A)

cosine_sim_df_A = pd.DataFrame(cosine_sim_A, index = data['상품명'], columns = data['상품명'])
print(cosine_sim_df_A.shape) # (item 개수 x item 개수)가 맞는지 확인
cosine_sim_df_A.head()

(449, 449)


상품명,베트남로부스타블루드래곤워시드g1,마일드블렌드,모카블렌드,하우스블렌드,헤이즐넛향커피,리치블렌드,아이스블렌드,에티오피아시다모g4,콜롬비아슈프리모후일라,브라질세하도ny-217-18,...,(에스프레소)디카페인 아티틀란 SHB 마운틴워터 과테말라,Campfire (캠프파이어),에티오피아 시다마 파피초 내추럴,온두라스 엘 사나테 파카스 워시드,콜롬비아 수프리모 후일라 산 비센테 라 플라타,나비,케냐 AA TOP 키안두 스페셜티 풀리워시드,다크우드,윈터 블렌드,인도네시아 만델링 G1 트리플픽
상품명,,,,,,,,,,,,,,,,,,,,,
베트남로부스타블루드래곤워시드g1,1.000000,0.467362,0.465062,0.335665,0.395433,0.372104,0.475149,0.454559,0.441895,0.458547,...,0.329883,0.450029,0.397348,0.461422,0.432176,0.504066,0.401579,0.480134,0.338068,0.384638
마일드블렌드,0.467362,1.000000,0.521211,0.521020,0.439053,0.749381,0.509171,0.581521,0.497348,0.475341,...,0.367755,0.465081,0.351264,0.497511,0.517945,0.541582,0.456888,0.530802,0.354424,0.357173
모카블렌드,0.465062,0.521211,1.000000,0.544247,0.432590,0.640808,0.736223,0.501032,0.492963,0.477539,...,0.355745,0.476452,0.304749,0.480996,0.483171,0.521126,0.439665,0.525177,0.367481,0.348059
하우스블렌드,0.335665,0.521020,0.544247,1.000000,0.325431,0.644131,0.492656,0.357291,0.365109,0.359793,...,0.249702,0.369662,0.148020,0.327732,0.337805,0.330241,0.289613,0.365167,0.306777,0.270012
헤이즐넛향커피,0.395433,0.439053,0.432590,0.325431,1.000000,0.360147,0.422596,0.411381,0.423571,0.410556,...,0.316247,0.405936,0.286211,0.414796,0.414222,0.439774,0.374953,0.434588,0.317954,0.299046


In [62]:
# 방법 B
cosine_sim_B = cosine_similarity(item_matrix_B)

cosine_sim_df_B = pd.DataFrame(cosine_sim_B, index = data['상품명'], columns = data['상품명'])
print(cosine_sim_df_B.shape) # (item 개수 x item 개수)가 맞는지 확인
cosine_sim_df_B.head()

(449, 449)


상품명,베트남로부스타블루드래곤워시드g1,마일드블렌드,모카블렌드,하우스블렌드,헤이즐넛향커피,리치블렌드,아이스블렌드,에티오피아시다모g4,콜롬비아슈프리모후일라,브라질세하도ny-217-18,...,(에스프레소)디카페인 아티틀란 SHB 마운틴워터 과테말라,Campfire (캠프파이어),에티오피아 시다마 파피초 내추럴,온두라스 엘 사나테 파카스 워시드,콜롬비아 수프리모 후일라 산 비센테 라 플라타,나비,케냐 AA TOP 키안두 스페셜티 풀리워시드,다크우드,윈터 블렌드,인도네시아 만델링 G1 트리플픽
상품명,,,,,,,,,,,,,,,,,,,,,
베트남로부스타블루드래곤워시드g1,1.000000,0.814668,0.807076,0.735634,0.807096,0.751553,0.851773,0.831245,0.809581,0.843107,...,0.801280,0.726936,0.652858,0.843590,0.781942,0.818961,0.786814,0.793655,0.723220,0.748852
마일드블렌드,0.814668,1.000000,0.986915,0.892920,0.977757,0.935762,0.983444,0.986573,0.994176,0.939107,...,0.974642,0.819684,0.751410,0.977201,0.976440,0.952531,0.976728,0.957335,0.827278,0.831809
모카블렌드,0.807076,0.986915,1.000000,0.937366,0.959106,0.963804,0.996484,0.982220,0.981057,0.939280,...,0.938645,0.836014,0.649026,0.940589,0.949627,0.912502,0.935755,0.943005,0.853964,0.838538
하우스블렌드,0.735634,0.892920,0.937366,1.000000,0.911175,0.965798,0.934070,0.884541,0.917602,0.893700,...,0.832027,0.819129,0.398101,0.809335,0.838438,0.730256,0.778413,0.828043,0.900284,0.821496
헤이즐넛향커피,0.807096,0.977757,0.959106,0.911175,1.000000,0.929312,0.955733,0.948496,0.991412,0.949747,...,0.981381,0.837723,0.716892,0.953984,0.957488,0.905668,0.938567,0.917772,0.868996,0.847337


## Content-Based Recommendation
타깃 상품을 입력하면 그와 유사한(피쳐들 간 코사인 유사도 값이 높은) 아이템을 찾아 추천

In [63]:
def get_recommendation_result(target_name, matrix, items, k=10):
    recom_idx = matrix.loc[:, target_name].values.reshape(1, -1).argsort()[:, ::-1].flatten()[1:k+1]
    
    recom_name = items.iloc[recom_idx, :]['상품명'].values
    recom_roastery = items.iloc[recom_idx, :]['로스터리'].values
    cosine_similarity = matrix.iloc[recom_idx][target_name].values
    recom_df = items.iloc[recom_idx, :]

    target_name_list = np.full(len(range(k)), target_name)
    target_roastery_list = np.full(len(range(k)), items[items['상품명'] == target_name]['로스터리'].values)

    recommendation_result = {
        'target_name':target_name_list,
        'target_roastery':target_roastery_list,
        'recom_name' : recom_name,
        'recom_roastery' : recom_roastery,
        'cosine_similarity': cosine_similarity
    }

    return pd.DataFrame(recommendation_result), recom_df

In [64]:
# 방법 A
result_A, recom_df_A = get_recommendation_result('베트남로부스타블루드래곤워시드g1', cosine_sim_df_A, data)
result_A

,target_name,target_roastery,recom_name,recom_roastery,cosine_similarity
0,베트남로부스타블루드래곤워시드g1,콩스콩스,베트남 블루드래곤 로부스타 워시드 G1,레스트빈,0.708593
1,베트남로부스타블루드래곤워시드g1,콩스콩스,바닐라향커피,콩스콩스,0.638547
2,베트남로부스타블루드래곤워시드g1,콩스콩스,보이져,스티머스 커피팩토리,0.622500
3,베트남로부스타블루드래곤워시드g1,콩스콩스,과테말라 SHB 스위스워터 디카페인,리프커피,0.598788
4,베트남로부스타블루드래곤워시드g1,콩스콩스,케냐 뚱구리 AA,영앤도터스,0.592327
5,베트남로부스타블루드래곤워시드g1,콩스콩스,콜롬비아블렌드,콩스콩스,0.587881
6,베트남로부스타블루드래곤워시드g1,콩스콩스,베러베스트 너티 블랜드,베러베스트,0.580794
7,베트남로부스타블루드래곤워시드g1,콩스콩스,멕시코 알투라 SHG 워시드,레스트빈,0.579716
8,베트남로부스타블루드래곤워시드g1,콩스콩스,날아올라,나무사이로,0.560884
9,베트남로부스타블루드래곤워시드g1,콩스콩스,에티오피아 시다모 물루게타 문타샤 워시드,카페플롬,0.552418


In [65]:
# 방법 B
result_B, recom_df_B = get_recommendation_result('베트남로부스타블루드래곤워시드g1', cosine_sim_df_B, data)
result_B

,target_name,target_roastery,recom_name,recom_roastery,cosine_similarity
0,베트남로부스타블루드래곤워시드g1,콩스콩스,베트남 블루드래곤 로부스타 워시드 G1,레스트빈,0.907838
1,베트남로부스타블루드래곤워시드g1,콩스콩스,아일리쉬 향커피,레스트빈,0.871771
2,베트남로부스타블루드래곤워시드g1,콩스콩스,아일리쉬향커피,콩스콩스,0.871771
3,베트남로부스타블루드래곤워시드g1,콩스콩스,콜롬비아블렌드,콩스콩스,0.866093
4,베트남로부스타블루드래곤워시드g1,콩스콩스,케냐 AA,커피창고,0.861153
5,베트남로부스타블루드래곤워시드g1,콩스콩스,로얄 블렌드,레스트빈,0.859291
6,베트남로부스타블루드래곤워시드g1,콩스콩스,아이스 블렌드,레스트빈,0.859291
7,베트남로부스타블루드래곤워시드g1,콩스콩스,로얄블렌드,콩스콩스,0.859291
8,베트남로부스타블루드래곤워시드g1,콩스콩스,브라질 옐로우 버번,커피창고,0.852244
9,베트남로부스타블루드래곤워시드g1,콩스콩스,아이스블렌드,콩스콩스,0.851773


In [66]:
# 타깃 상품 특징 살펴보기
data[data['상품명'] == '베트남로부스타블루드래곤워시드g1']

,상품명,Cupping Note 향미,Cupping Note 산미,Cupping Note 단맛,Cupping Note 바디감,Cupping Note 밸런스,Roasting Point,origin,text,로스터리,평점,리뷰 수,keywords_set,keywords_sentence,Keywords Jaccard Similarity
0,베트남로부스타블루드래곤워시드g1,7.0,6,7,8,7.0,5.5,베트남,로부스타 특유의 곡물의 구수한 향미와 은은한 단 맛이 느껴지는 커피입니다. 수세식(...,콩스콩스,4.35,255,"{워시드, 방식, 가공, 향미, 곡물, 특유, 선사}",향미 가공 워시드 선사 방식 곡물 특유,1.0


In [67]:
recom_df_A # 방법 A의 결과

,상품명,Cupping Note 향미,Cupping Note 산미,Cupping Note 단맛,Cupping Note 바디감,Cupping Note 밸런스,Roasting Point,origin,text,로스터리,평점,리뷰 수,keywords_set,keywords_sentence,Keywords Jaccard Similarity
87,베트남 블루드래곤 로부스타 워시드 G1,7.0,6,7,8,7.0,5.5,베트남,은은한 단맛이 느껴지는 커피입니다. 수세식으로 가공되어 보다 균일하고 깔끔한 에프터...,레스트빈,4.33,6,"{단맛, 가공, 선사}",가공 단맛 선사,0.250000
11,바닐라향커피,8.0,6,6,7,6.0,5.5,베트남,아로아향처럼 기분을 좋게 만들며 입 안에 감도는 달콤한 맛과 부드러우면서도 고소하 ...,콩스콩스,3.35,20,{},,0.000000
241,보이져,7.0,8,8,6,NaN,5.0,"온두라스, 에티오피아, 에티오피아",ㅁㅜㄹㅡㅇㅣㄱㅇㅡㄴ ㄱㅘㅇㅣㄹㅇㅢ ㅅㅏㄴㅁㅣ가 ㄷㅏㄹㅋㅗㅁ해요. ㅍㅜㅇㅂㅜㅎㅏㄴ ㄲ...,스티머스 커피팩토리,4.33,6,{},,0.000000
386,과테말라 SHB 스위스워터 디카페인,7.0,4,8,8,NaN,6.0,GUATEMALA,"곡물향, 달콤한 카라멜향, 쌉쌀하면서 부드러운 맛",리프커피,4.33,6,"{곡물, 카라멜}",곡물 카라멜,0.125000
301,케냐 뚱구리 AA,7.0,8,8,6,NaN,2.0,케냐,가공소 / 뚱구리 Thunguri Washing Station 생산자 / 키비리기 ...,영앤도터스,4.33,6,"{품종, 워시드, 가공, 로스팅}",가공 워시드 로스팅 품종,0.222222
17,콜롬비아블렌드,7.0,7,7,7,8.0,5.5,"콜롬비아, 브라질, 베트남",구수한 향미와 연하고 부드러운 맛을 느낄수 있습니다,콩스콩스,3.88,8,{향미},향미,0.142857
55,베러베스트 너티 블랜드,6.0,4,8,8,NaN,4.5,"과테말라, 브라질",고소한 맛에 고소한 맛을 더한 원두! 적은 산미에 곡물류의 고소함과 견과류의 고소함...,베러베스트,4.33,6,"{곡물, 산미, 견과}",견과 산미 곡물,0.111111
107,멕시코 알투라 SHG 워시드,7.0,7,7,8,7.0,5.5,멕시코,"가벼운 바디감과 낮은 산미, 커피 특유의 단맛으로 백포도주를 연상케하여 처음으로 커...",레스트빈,4.33,6,"{처음, 바디, 산미, 시작, 단맛, 연상, 특유}",단맛 처음 시작 산미 바디 연상 특유,0.076923
419,날아올라,7.0,8,8,8,NaN,2.0,에티오피아,"꽃처럼, 향수처럼, 아름다운 커피",나무사이로,4.33,6,{},,0.000000
303,에티오피아 시다모 물루게타 문타샤 워시드,7.0,8,10,6,NaN,2.0,에티오피아,복합적인 맛을 지닌 깔끔한 에티오피아 워시드입니다.,카페플롬,4.33,6,"{복합, 에티오피아, 워시드, 깔끔}",에티오피아 복합 깔끔 워시드,0.100000


**방법 A - 즉 TF-IDF로 텍스트를 처리한 경우**
- 타깃 상품에서 언급된 키워드들을 포함하는 상품들을 많이 추천해주고 있으나 키워드가 아예 없는 경우도 있다.
- 즉 키워드 유사도 뿐만 아니라 커핑노트 관련 컬럼들과 유사도가 높은 아이템들도 많이 추천해주는 것 같다.
- 이 결과가 과연 좋은 건진 모르겠지만, 아래 B 결과에 비해서 타깃 아이템과 조금 덜 비슷한 대신 다양한 아이템을 추천해주는 느낌이다.

In [68]:
recom_df_B # 방법 B의 결과

,상품명,Cupping Note 향미,Cupping Note 산미,Cupping Note 단맛,Cupping Note 바디감,Cupping Note 밸런스,Roasting Point,origin,text,로스터리,평점,리뷰 수,keywords_set,keywords_sentence,Keywords Jaccard Similarity
87,베트남 블루드래곤 로부스타 워시드 G1,7.0,6,7,8,7.0,5.5,베트남,은은한 단맛이 느껴지는 커피입니다. 수세식으로 가공되어 보다 균일하고 깔끔한 에프터...,레스트빈,4.33,6,"{단맛, 가공, 선사}",가공 단맛 선사,0.250000
91,아일리쉬 향커피,8.0,6,6,7,6.0,5.5,베트남,베트남 원두를 베이스로 아일리쉬 향을 가미하여 아일리쉬 특유의 위스키함과 크림 향미...,레스트빈,4.33,6,"{베이스, 특유, 향미, 베트남, 가미, 크림}",향미 크림 가미 베이스 베트남 특유,0.181818
12,아일리쉬향커피,8.0,6,6,7,6.0,5.5,베트남,베트남 원두를 베이스로 아일리쉬 향을 가미하여 아일리쉬 특유의 위스키향과 크림 향미...,콩스콩스,4.07,15,"{베이스, 특유, 향미, 베트남, 가미, 크림}",향미 크림 가미 베이스 베트남 특유,0.181818
17,콜롬비아블렌드,7.0,7,7,7,8.0,5.5,"콜롬비아, 브라질, 베트남",구수한 향미와 연하고 부드러운 맛을 느낄수 있습니다,콩스콩스,3.88,8,{향미},향미,0.142857
71,케냐 AA,6.0,6,6,6,NaN,5.5,NaN,케냐 AA는 자몽 같은 과일 산미와 당밀 특유의 향미와 단맛이 은은하게 입안 가득 ...,커피창고,4.33,6,"{당밀, 캬라멜, 산미, 과일, 단맛, 특유, 향미, 케냐, 여운, 자몽, 풍미}",케냐 캬라멜 단맛 향미 여운 산미 당밀 자몽 풍미 특유 과일,0.125000
102,로얄 블렌드,6.0,6,6,7,6.0,5.5,"브라질, 인도네시아, 베트남","고소한 향미와 맛, 쌉싸름한 맛이 어울러져 누구나 접할 수 있는 블렌딩 커피입니다.",레스트빈,4.33,6,"{블렌딩, 향미, 쌉싸름}",쌉싸름 향미 블렌딩,0.111111
100,아이스 블렌드,6.0,6,6,7,6.0,5.5,"콜롬비아, 브라질, 베트남",더운 여름 커피의 진한 향미와 맛을 시원하게 즐길 수 있는 아이스 블렌딩 커피입니다.,레스트빈,4.33,6,"{블렌딩, 향미, 아이스}",블렌딩 아이스 향미,0.111111
19,로얄블렌드,6.0,6,6,7,6.0,5.5,"브라질, 베트남","고소한 향미와 맛, 쌉싸름한 맛이 어우러져 누구나 접할 수 있는 블렌딩 커피입니다.",콩스콩스,0.00,0,"{블렌딩, 향미, 쌉싸름}",쌉싸름 향미 블렌딩,0.111111
76,브라질 옐로우 버번,6.0,4,6,6,NaN,5.0,NaN,옐로우라는 이름에 걸맞게 커피체리의 색이 노란색을 가진 것이 옐로우 버번입니다. 맛...,커피창고,4.33,6,"{농후, 산미, 견과, 매력, 감귤, 향미, 체리}",견과 산미 체리 향미 농후 매력 감귤,0.076923
6,아이스블렌드,6.0,6,6,7,6.0,6.0,"콜롬비아, 브라질, 베트남",더운 여름 커피의 진한 향미와 맛을 시원하게 즐기실 수 있는 아이스 블렌딩 커피입니다.,콩스콩스,5.00,1,"{블렌딩, 향미, 아이스}",블렌딩 아이스 향미,0.111111


**방법 B, 즉 자카드 유사도로 텍스트 데이터를 활용한 경우**
- 타깃 아이템과 훨씬 더 비슷한 아이템들을 추천해준다. 대부분 콩스콩스와 구성이 비슷했던 로스터리와 상품명을 가진 아이템들이 추천되었다. 
- 키워드가 전체적으로 배우 비슷하며 (거의 모두 '향미'라는 키워드를 가짐) 커핑 노트와 로스팅 포인트도 거의 동일하다. 코사인 유사도 값 또한 압도적으로 높다.
- 유사한 아이템을 추천하는 데에는 방법 A보다는 B가 효과적이라고 판단된다.

그리고 전체적으로 보았을 때, **top 5개** 아이템까지만 추천해줘도 충분할 것 같다.